In [86]:

import os
import pickle
import pandas as pd
import sys
import sys
from memory_profiler import profile
import pandas as pd
import numpy as np
sys.path.append('/home/ak/Documents/Research/PaperCode/MultiKernelLearning')
sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
import jsonpickle
from collections import defaultdict
from sklearn.svm import SVC
from collections import OrderedDict
import os
from sklearn.metrics import accuracy_score
import torch


from fileutils import new_feature_utils as nfu
from fileutils.new_feature_utils import CreateMarketFeatures
import multiprocessing
import mkl_data_processing as mkldp
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
import matplotlib as mpl
import matplotlib.pyplot as plt
styleFiles = os.listdir(mpl.get_configdir())
styleFileIdx = 0

plt.style.use(os.path.join(mpl.get_configdir(), styleFiles[styleFileIdx]))

In [87]:
def open_pickle_filepath(pickle_file):
    pickle_to_file = pickle.load(open(pickle_file, "rb"), encoding='latin1')

    return pickle_to_file

In [88]:
files_location = '/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSPredictions/SimpleMKL/'
symbols = sorted(os.listdir(files_location))
#processed results path
processed_location = '/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSProcessed'

In [121]:
symbolIDX =  26 #pick a symbol
label_idx = 4 #pick a label
label_string= 'Label_idx:_'+str(label_idx)
print('you are doing symbol', symbols[symbolIDX], 'and label', label_string)
# now we take out the files for the symbol 
symbol_path = os.path.join(files_location,str(sorted(symbols)[symbolIDX]))
symbolFiles = os.listdir(os.path.join(files_location,str(sorted(symbols)[symbolIDX])))
# now isolate all the label+ symbol combos
symbolLabelFiles = [f for f in symbolFiles if str(label_string) in f]
path_to_store = os.path.join(processed_location, symbols[symbolIDX])
number_of_files = len(symbolLabelFiles) 

# this is the number of files that correspond to the label/symbol combo

print('total files for the symbol:', len(symbolFiles))
print('and for this label')
print(number_of_files)

you are doing symbol RR.L and label Label_idx:_4
total files for the symbol: 417
and for this label
417


In [122]:
path_to_store

'/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSProcessed/RR.L'

In [123]:
dates_chunks_keys = dict()
chunks_list =list()
keys_list = list()

for file_to_load_idx in range(0,number_of_files):
    # go through all of the files that are a combination of symbols and labels
    file_to_load_path = os.path.join(os.path.join(files_location, str(symbols[symbolIDX])), symbolLabelFiles[file_to_load_idx])
    if os.path.getsize(file_to_load_path)> 0:
        # extract the key - these are the symbol feature path keys in the  main code
        file_keys = list(open_pickle_filepath(file_to_load_path).keys()) # keys
        keys_list.append(file_keys[0])
        dates_chunks_keys[file_to_load_idx] = file_keys
        chunks_list.append(open_pickle_filepath(file_to_load_path)[file_keys[0]])
    else:
        continue

        

In [124]:
dates_df = pd.DataFrame((np.unique(list(dates_chunks_keys.values()))))
list_of_dfs_across_days = list()
for chunk_list_item in range(0,len(chunks_list)-1):
    chunk_list_item_dict = chunks_list[chunk_list_item]
    chunk_list_item_keys = list(chunk_list_item_dict.keys())
    list_of_dicts=list()
    for key in chunk_list_item_keys:
        list_of_dicts.append(pd.DataFrame(chunk_list_item_dict[key]))
    list_of_dfs_across_days.append(pd.concat(list_of_dicts,axis=1).median(axis=1))


In [125]:
list_of_dfs_across_days

[Hamming Loss    0.18
 accuracy        0.82
 f1- macro       0.45
 f1- micro       0.82
 f1- weighted    0.74
 precision       0.67
 recall          0.82
 dtype: float64, Hamming Loss    0.18
 accuracy        0.82
 f1- macro       0.45
 f1- micro       0.82
 f1- weighted    0.74
 precision       0.68
 recall          0.82
 dtype: float64, Hamming Loss    0.16
 accuracy        0.84
 f1- macro       0.46
 f1- micro       0.84
 f1- weighted    0.77
 precision       0.71
 recall          0.84
 dtype: float64, Hamming Loss    0.16
 accuracy        0.84
 f1- macro       0.46
 f1- micro       0.84
 f1- weighted    0.76
 precision       0.70
 recall          0.84
 dtype: float64, Hamming Loss    0.180
 accuracy        0.820
 f1- macro       0.450
 f1- micro       0.820
 f1- weighted    0.740
 precision       0.675
 recall          0.820
 dtype: float64, Hamming Loss    0.18
 accuracy        0.82
 f1- macro       0.45
 f1- micro       0.82
 f1- weighted    0.74
 precision       0.68
 recall    

In [126]:
df =pd.concat(list_of_dfs_across_days, axis =1)
df.median(axis=1)

Hamming Loss    0.18
accuracy        0.82
f1- macro       0.45
f1- micro       0.82
f1- weighted    0.74
precision       0.68
recall          0.82
dtype: float64

In [127]:
#df.to_csv('file_name.csv
location_for_df = os.path.join(path_to_store, symbols[symbolIDX] +"_Label_"+str(label_idx)+"_all_chunks.csv")
df.to_csv(location_for_df, index= True)
location_for_dates_df = os.path.join(path_to_store, symbols[symbolIDX] +"_Label_"+str(label_idx)+"_dates.csv")
dates_df.to_csv(location_for_dates_df, index = True)

In [128]:
df_test = pd.read_csv(location_for_df, index_col=0)
df_test.median(axis=1)

Hamming Loss    0.18
accuracy        0.82
f1- macro       0.45
f1- micro       0.82
f1- weighted    0.74
precision       0.68
recall          0.82
dtype: float64

# to do:
1. write a loop to go through all the labels for each symbol and store them in a seperate file
2. concentrate only on the symbols we know we have labels
3. pull the file from Single Kernel and compare them.
4. re-do the labels and the plots for the symbols that you are picking and describe them

In [97]:

# Path
def create_path(parent_dir, symbol):
    path = os.path.join(parent_dir, symbol)
    if os.path.isdir(path)==False:
        # Create the directory
    
        os.mkdir(path)
        print("Directory '%s' created" %path)
    else:
        print("Directory Exist")
        pass

In [98]:
symbols[symbolIDX]

'REL.L'

In [99]:
# for symbolIDX in range(0, len(symbols)):
#     create_path(processed_location, symbols[symbolIDX])

In [100]:
symbols_to_load = os.listdir(processed_location)

In [105]:
list_files = list()
string = str('_Label_2')
for idx in range(0,40):
    files =os.listdir(os.path.join(processed_location, symbols_to_load[idx]))
    list_files.append([f for f in files if string in f])

        

In [106]:
list_files

[[],
 [],
 [],
 ['BLT.L_Label_2_all_chunks.csv', 'BLT.L_Label_2_dates.csv'],
 ['KGF.L_Label_2_all_chunks.csv', 'KGF.L_Label_2_dates.csv'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['CEY.L_Label_2_all_chunks.csv', 'CEY.L_Label_2_dates.csv'],
 [],
 [],
 ['ITV.L_Label_2_all_chunks.csv', 'ITV.L_Label_2_dates.csv'],
 [],
 [],
 ['APF.L_Label_2_all_chunks.csv', 'APF.L_Label_2_dates.csv'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['CPG.L_Label_2_dates.csv', 'CPG.L_Label_2_all_chunks.csv'],
 ['BARC.L_Label_2_dates.csv', 'BARC.L_Label_2_all_chunks.csv'],
 [],
 [],
 [],
 [],
 [],
 []]

In [66]:
label_1_files = ['APF.L_Label_1_all_chunks.csv', 'CCL.L_Label_1_all_chunks.csv','AV.L_Label_1_all_chunks.csv']
label_5_files = ['PSON.L_Label_5_all_chunks.csv', 'PRU.L_Label_5_all_chunks.csv',
                 'AV.L_Label_5_all_chunks.csv', 'APF.L_Label_5_all_chunks.csv']

label_4_files = ['BLT.L_Label_4_all_chunks.csv', 
                 'CEY.L_Label_4_all_chunks.csv',
                 'ITV.L_Label_4_all_chunks.csv',
                 'APF.L_Label_4_all_chunks.csv',
                 'AZN.L_Label_4_all_chunks.csv',
                 'CPG.L_Label_4_all_chunks.csv',
                 'AV.L_Label_4_all_chunks.csv'
                ]

label_2_files = ['BLT.L_Label_2_all_chunks.csv', 
                 'KGF.L_Label_2_all_chunks.csv',
                'CEY.L_Label_2_all_chunks.csv',
                 'CPG.L_Label_2_all_chunks.csv',
                 'BARC.L_Label_2_all_chunks.csv',
                 'ITV.L_Label_2_all_chunks.csv',
                 'APF.L_Label_2_all_chunks.csv' ]

label_3_files =['LGEN.L_Label_3_all_chunks.csv',
                'APF.L_Label_3_all_chunks.csv',
                'AV.L_Label_3_all_chunks.csv',
                'MAB.L_Label_3_all_chunks.csv',
                'LAND.L_Label_3_all_chunks.csv',
               'LLOY.L_Label_3_all_chunks.csv',
               'RBS.L_Label_3_all_chunks.csv']



In [25]:
row_labels = ['Hamming Loss',
 'accuracy',
 'f1- macro',
 'f1- micro',
 'f1- weighted',
 'precision',
 'recall']

In [67]:
label_3_dfs = list()
label_3_dicts = dict()

for idx in range(0, len(label_3_files)):
    symbol = label_3_files[idx].split("_")[0]
    
    file_to_load = \
    pd.read_csv(os.path.join(processed_location, symbol, label_3_files[idx])).median(axis=1)
    label_3_dfs.append(file_to_load)
    file_to_load.index = row_labels
    label_3_dicts[symbol] = file_to_load

In [68]:
label_3_df = pd.concat(label_3_dfs, axis =1).median(axis =1)

In [72]:
pd.DataFrame(label_3_dfs).T

,0,1,2,3,4,5,6
Hamming Loss,0.66,0.515,0.62,0.53,0.660,0.16,0.62
accuracy,0.34,0.485,0.38,0.47,0.340,0.84,0.38
f1- macro,0.25,0.290,0.28,0.25,0.230,0.46,0.26
f1- micro,0.34,0.485,0.38,0.47,0.340,0.84,0.38
f1- weighted,0.25,0.330,0.31,0.35,0.245,0.77,0.30
precision,0.24,0.250,0.30,0.37,0.240,0.71,0.30
recall,0.34,0.485,0.38,0.47,0.340,0.84,0.38


In [28]:
df_concatenated = pd.concat([label_1_df, label_2_df,
                             label_3_df, label_4_df,label_5_df], axis=1)
df_concatenated.columns = [ 'Label 1','Label 2','Label 3','Label 4', 'Label 5' ]

NameError: name 'label_1_df' is not defined

In [29]:
df_concatenated

NameError: name 'df_concatenated' is not defined

In [ ]:
print(label_2_dicts['BLT.L'].to_latex())

In [ ]:
print(pd.DataFrame.from_dict(label_1_dicts).to_latex()) #.drop(columns=['APF.L']).to_latex())

In [ ]:
label_6_df